In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [4]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [6]:
import os

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

In [7]:
client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)

In [8]:
vector_store_page = QdrantVectorStore(
    client=client,
    collection_name="db-book-page",
    embedding=embeddings,
)

In [10]:
from langchain_core.documents import Document
def combine_documents(documents: list[Document]) -> str:
    return "\n\n".join([document.page_content for document in documents])


In [14]:
SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TAREA>
Tu tarea es responder la pregunta del usuario.
</TAREA>

<RESTRICCIONES>
- Solo responde la pregunta del usuario tomando como contexto lo provisto en <CONTEXTO>.
</RESTRICCIONES>

<CONTEXTO>
{context}
</CONTEXTO>

"""

USER_PROMPT = """
user question: {user_request}
rephrased user question: {rephrased_request}
"""

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = ChatPromptTemplate([
    ("ai", SYSTEM_PROMPT),
    ("user", USER_PROMPT)
])


In [11]:
SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TASK>
Tu tarea es refrasear la solicitud del usuario para genera una solicitud refraseada.

- Puedes corregir los errores gramaticales
- Puedes mejorar la semántica y orden léxico de la palabras para un mejor entendimiento
</TASK>
"""

USER_PROMPT = """{user_request}"""

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

rephrase_prompt = ChatPromptTemplate([
    SystemMessage(content=SYSTEM_PROMPT),
    ("user", USER_PROMPT)
])


In [15]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | RunnableLambda(lambda x: x.content)
    } 
    | RunnablePassthrough() 
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "context": itemgetter("rephrased_request") | vector_store_page.as_retriever(search_kwargs={"k": 10}) | RunnableLambda(combine_documents),
    }
    | qa_prompt 
    | llm
    | RunnableLambda(lambda x: x.content)
)


In [16]:
simple_chatbot.invoke({"user_request": "quien es susan fletcher"})

'Susan Fletcher es la criptógrafa estrella de la ultrasecreta Agencia de Seguridad Nacional (NSA). Es una mujer muy inteligente, con un Cociente de Inteligencia de 170, y es la jefa de la División de Criptografía de la NSA. A lo largo de la historia, se enfrenta a situaciones de alta tensión, incluyendo la interceptación de un código indescifrable y la amenaza de un asesino a sueldo. Además, está emocionalmente involucrada con David Becker, su prometido, quien también juega un papel crucial en la trama.'